In [1]:
!pip install underthesea

Defaulting to user installation because normal site-packages is not writeable
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached scikit_learn-1.5.2-cp312-cp312-win_amd64.whl.metadata (13 kB)
     ---------------------------------------- 0.0/560.4 kB ? eta -:--:--
     -------------------------------------- 560.4/560.4 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'


ERROR: Exception:
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2032.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_internal\cli\base_command.py", line 105, in _run_wrapper
    status = _inner_run()
             ^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2032.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_internal\cli\base_command.py", line 96, in _inner_run
    return self.run(options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2032.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_internal\cli\req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2032.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_internal\commands\install.py", line 379, in run
    requirement_set = resolver

In [2]:
import pandas as pd 
import os
from string import punctuation 
from underthesea import word_tokenize
import re
from time import time

ImportError: Unable to import required dependencies:
numpy: No module named 'numpy'

In [3]:
# evaluate quantile 
def quantile(df,quantiles): 
    '''
    Input: 
    x: considered corpus/context dataframe (cid,text)
    quantiles is the range of considered quantiles
    Output: 
    the dictionary: key: cid, value: context
    '''
    if df == 'corpus':
        sorted_df = corpus.sort_values(by='count',ascending=True)
        len_df = len(sorted_df)
        result = []
        for i,percent in enumerate(quantiles): 
            if i == 0: 
                distribution = int(len_df*percent)
                temp = dict(zip(sorted_df.loc[:distribution]['cid'],sorted_df.iloc[:distribution]['text']))
            elif i == len(quantiles)-1: 
                distribution = int(len_df*percent)
                temp = dict(zip(sorted_df.loc[distribution:]['cid'],sorted_df.iloc[distribution:]['text']))
            else: 
    #            #Consider the data in rage of quantiles[i-1] and quantiles[i]
                bef_disb = int(len_df*quantiles[i-1])
                aft_disb = int(len_df*quantiles[i])
                temp = dict(zip(sorted_df.loc[bef_disb:aft_disb]['cid'],sorted_df.iloc[bef_disb:aft_disb]['text'] ))
            result.append(temp)
        return result
    elif df == 'train': 
        sorted_df = train.sort_values(by='count_context',ascending=True)
        len_df = len(sorted_df)
        result = []
        for i,percent in enumerate(quantiles): 
            if i == 0: 
                distribution = int(len_df*percent)
                temp = dict(zip(sorted_df.iloc[:distribution]['cid'],sorted_df.iloc[:distribution]['context']))
            elif i == len(quantiles)-1: 
                distribution = int(len_df*percent)
                temp = dict(zip(sorted_df.iloc[distribution:]['cid'],sorted_df.iloc[distribution:]['context']))
            else: 
    #            #Consider the data in rage of quantiles[i-1] and quantiles[i]
                bef_disb = int(len_df*quantiles[i-1])
                aft_disb = int(len_df*quantiles[i])
                temp = dict(zip(sorted_df.iloc[bef_disb:aft_disb]['cid'],sorted_df.iloc[bef_disb:aft_disb]['context'] ))
            result.append(temp)
        return result
            
    
    

# EDA corpus

In [ ]:
corpus = pd.read_csv('/kaggle/input/legal-document/corpus.csv')
corpus.head(5)
# text: Một đoạn văn bản pháp luật bất kỳ (dạng string) - documents
# cid: Id của đoạn văn bản đó trong corpus (dạng int)


In [ ]:
corpus.info()

In [ ]:
# count word in text 
corpus['count'] = corpus['text'].map(lambda x: len(x.split()))
corpus['count'].describe()

In [ ]:
print('The length of the longest texts is',corpus['text'].apply(len).min())
print('The length of the shortest texts is',corpus['text'].apply(len).max())
# The length of the longest texts is 252881
# The length of the shortest texts is 4
# Filter all sentences which have above 2000 words to chunking or gen title

In [ ]:
len(corpus)
# 261597

In [ ]:
quantile('corpus',[0.25,0.5,0.75])

In [ ]:
corpus[corpus['count'] < 4]

In [11]:
# remove unvaluable texts
corpus = corpus[corpus['count'] >= 4]

In [ ]:
corpus.describe()

# EDA train

In [ ]:
train = pd.read_csv('/kaggle/input/legal-document/train.csv')
print(train.head(5))
print('The length of train dataframe:', len(train))    #119456

'''
Là tập dữ liệu đã được gán nhãn, dùng để huấn luyện mô hình. 
Tập này gồm 119456 cặp truy vấn và văn bản tương ứng.

- question: Dạng văn bản của câu hỏi (dạng string)
- qid: Mã id của câu hỏi (viết tắt của question_id, dạng string)
- context: Các đoạn văn bản luật pháp liên quan (dạng list)
- cid: Mã id của các đoạn văn bản pháp luật trong corpus có liên quan tới câu hỏi (viết tắt của context_id, dạng list)
'''

In [ ]:
print('sample question',train.iloc[0,0])
print('sample answer',train.iloc[0,1])

In [ ]:
train.describe()

In [ ]:
train['count_context'] = train['context'].map(lambda x: len(x.split()))
train.describe()

In [ ]:
quantile('train',[0.25])

In [ ]:
import json 
with open('/kaggle/input/legal-document/qid_to_cid.json','r') as f: 
    qid_to_cid = json.load(f)
qid_to_cid
# train

In [ ]:
# validate cid between train and corpus 
train_cid_not_in_corpus = []
train_cid = qid_to_cid.values()
for i,lst_cid in enumerate(train_cid): 
    for cid in lst_cid:
        if cid not in corpus['cid'].unique():   
            train_cid_not_in_corpus.append(train.iloc[i]['qid'])
train_cid_not_in_corpus[:5]

In [ ]:
len(train_cid_not_in_corpus)

In [ ]:
corpus[corpus['cid'] == int(84570)]
# check

In [ ]:
for k,v in qid_to_cid.items():
    if v == [84570]: 
        print(train['context'][train['qid'] == int(k)])

There are 197 cids which don't belong to corpus, so drop these in train_data

In [ ]:
# remove special cases
train['context'][train['count_context']< 6]

In [ ]:
train = train[train['count_context'] >=6]

In [ ]:
train.describe()

In [ ]:
processed_qid2cid = {}
for i in len(train): 
    processed_qid2cid[train['qid']] = train['cid']

## Tiền xử lý dữ liệu 
- Triển khai kế hoạch thực nghiệm và đánh giá kết quả 
https://docs.google.com/spreadsheets/d/1PdgF9GImBaaAwthM_jlzSHjJAf_hS3pecE1a_IotVW4/edit?usp=sharing

In [ ]:
data_stopword = open('/kaggle/input/vietnamese-stopword/vietnamese-stopwords.txt').read().split('\n')
stopword = [i for i in data_stopword]

In [ ]:
class Data_Preprocessing: 
    def __init__(self,data,id): 
        self.data = data 
        self.id = id
        self.stopword = stopword
        self.punctuation = punctuation
    def _remove_stop_word(self,vocab): 
        result = []
        for i in vocab: 
            if i not in self.stopword: 
                result.append(i)
        return result
    def _remove_punctuation(self): 
        temp = []
        for doc in self.data: 
            doc = re.sub(rf'[{punctuation}]','',doc)
            temp.append(doc)
        return temp
    def preprocessing_(self,is_backup=False): 
        result = {}
        temp = self._remove_punctuation()
        for i in range(len(temp)): 
            vocab = []
            sens = word_tokenize(temp[i],format='text')
            sens = sens.split() 
            for w in sens: 
                if w not in vocab: 
                    vocab.append(w)
            vocab = list(set(vocab))
            vocab = self._remove_stop_word(vocab)
            if is_backup: 
                result[self.id[i]] = ' '.join(vocab)
            else:
                result[self.id[i]] = vocab
            
        return result

### Với tập question

In [ ]:
train_question = Data_Preprocessing(train['question'],train['qid'])

In [ ]:
vocab_train = train_question.preprocessing_()

In [ ]:
first_5_items = dict(list(vocab_train.items())[:5])
print(first_5_items)

### Với tập corpus

In [ ]:
corpus['text'].apply(len).max()
# 252881

In [ ]:
corpus['text'].apply(len).min()

In [ ]:
preprocess_corpus = Data_Preprocessing(corpus['text'],corpus['cid'])

In [ ]:
start_time = time()
preprocess_corpus = preprocess_corpus.preprocessing_(is_backup=True)
end_time = time()
print('Total time of processing corpus:', end_time - start_time)

In [ ]:
processed_corpus_df = pd.DataFrame({'id':preprocess_corpus.keys(),'text':preprocess_corpus.values()})

In [ ]:
processed_corpus_df['text'][:5]

In [ ]:
processed_corpus_df.to_csv('/kaggle/working/processed_corpus.csv')

In [ ]:
import os 
os.chdir(r'/kaggle/working')

In [ ]:
from IPython.display import FileLink 
FileLink(r'processed_corpus.csv')

Lọc ra các context có số lượng từ >= 2000 để xử lý theo 1 trong 3 hướng tiếp cận: 
1. Sử dụng gen title = LLM và đánh giá hiệu quả 
2. Sử dụng kỹ thuật chunking và đánh giá hiệu quả 
3. Trích lọc các câu chính nhất hoặc tổng quát trong context (benchmark to evaluate RAG - dspy)